In [1]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
import os

os.chdir(proj_root())

In [16]:
from coeditor.retrieval_model import RetrievalEditorModel
from coeditor.api import EditPredictionService, BasicQueryEditEncoder, BatchArgs

model_path = get_model_dir(False) / "coeditor-large-query-basic/checkpoint-136312"
model = RetrievalEditorModel.load(model_path)
model.to("cuda:1")
service = EditPredictionService(
    model, encoder=BasicQueryEditEncoder(max_ref_tks=512), 
    batch_args=BatchArgs(max_ref_tks=50*512, max_ref_dropout=0.0, shuffle_extra_ids=False),
)


/home/jiayi/Projects/SPOT/src/coeditor/retrieval_model.py:255: UserWarning: No extra args found, using default model setting.
  warnings.warn("No extra args found, using default model setting.")


In [23]:
service.suggest_edit(proj_root(), Path("src/coeditor/history.py"), (52, 0))

num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
len(input_tks) = 49
len(references) = 43
========== Predicted code change ==========
* Modified: 
    # path: coeditor.history/Deleted.as_char

    @dataclass
    class Deleted(Generic[T1]):
        @staticmethod
        def as_char():
            return "D"


In [12]:
service.tlogger.as_dataframe()


,name,count,avg_time,total_time
1,construct project edit,7,3.761061,26.327428
0,get target element,7,0.430467,3.013267
3,run model,3,0.717002,2.151007
2,encode edits,7,0.143082,1.001571
